FastText, introduced by Facebook's AI Research (FAIR), extends Word2Vec by incorporating subword information into word embeddings. This makes FastText particularly useful for handling out-of-vocabulary (OOV) words and morphologically rich languages.

#### Key Concepts of FastText
##### Subword Representations:
- Instead of treating each word as an atomic unit, FastText breaks words into n-grams (e.g., "apple" → <ap, app, ppl, ple, le> for n=3).
- The word embedding is the sum of its n-gram embeddings.

##### Training Objective:

- Like Word2Vec, FastText supports Skip-gram and CBOW architectures.

##### Advantages:
- Handles OOV words by **summing the embeddings of their subwords**.
- Improves performance on rare words by sharing information across similar subwords.


##### Implementation Steps
- Preprocess the text and build vocabulary.
- Generate subword n-grams.
- Train using the Skip-gram architecture.
- Learn embeddings for words and subwords.

In [1]:
import re
import numpy as np
from collections import defaultdict, Counter
import random

class FastText:
    def __init__(self, vector_size=50, window_size=2, learning_rate=0.01, epochs=10, min_n=3, max_n=6, negative_samples=5):
        """
        Initializes the FastText model.
        
        Parameters:
        - vector_size (int): Dimensionality of word vectors.
        - window_size (int): Context window size.
        - learning_rate (float): Learning rate for gradient descent.
        - epochs (int): Number of training iterations.
        - min_n (int): Minimum length of subword n-grams.
        - max_n (int): Maximum length of subword n-grams.
        - negative_samples (int): Number of negative samples for each positive sample.
        """
        self.vector_size = vector_size
        self.window_size = window_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.min_n = min_n
        self.max_n = max_n
        self.negative_samples = negative_samples
        self.vocab = {}
        self.word_to_index = {}
        self.index_to_word = {}
        self.subword_to_index = {}
        self.W = None  # Word vectors
        self.W_sub = None  # Subword vectors
    
    def preprocess(self, text):
        """
        Preprocesses the input text (tokenization, lowercasing).
        
        Parameters:
        - text (str): The raw text string.
        
        Returns:
        - tokens (list): A list of preprocessed tokens.
        """
        text = text.lower()
        text = re.sub(r'\W+', ' ', text)
        tokens = text.split()
        return tokens
    
    def build_vocab(self, corpus):
        """
        Builds the vocabulary and subword n-grams.
        
        Parameters:
        - corpus (list of str): A list of documents (each document is a string).
        """
        tokens = []
        for doc in corpus:
            tokens.extend(self.preprocess(doc))
        
        # Count word frequencies
        word_counts = Counter(tokens)
        
        # Build vocab and initialize word-to-index mappings
        self.word_to_index = {word: i for i, word in enumerate(word_counts)}
        self.index_to_word = {i: word for word, i in self.word_to_index.items()}
        self.vocab = set(self.word_to_index.keys())
        
        # Build subword n-grams
        subwords = set()
        for word in self.vocab:
            subwords.update(self.get_subwords(word))
        
        self.subword_to_index = {sub: i for i, sub in enumerate(subwords)}
        
        # Initialize embeddings
        vocab_size = len(self.vocab)
        subword_size = len(self.subword_to_index)
        self.W = np.random.rand(vocab_size, self.vector_size)  # Word embeddings
        self.W_sub = np.random.rand(subword_size, self.vector_size)  # Subword embeddings
    
    def get_subwords(self, word):
        """
        Generates subword n-grams for a given word.
        
        Parameters:
        - word (str): The input word.
        
        Returns:
        - subwords (set): A set of subword n-grams.
        """
        word = f"<{word}>"
        subwords = set()
        for n in range(self.min_n, self.max_n + 1):
            for i in range(len(word) - n + 1):
                subwords.add(word[i:i + n])
        return subwords
    
    def generate_training_data(self, corpus):
        """
        Generates training data (target-context pairs) for the Skip-gram model.
        
        Parameters:
        - corpus (list of str): A list of documents.
        
        Returns:
        - training_data (list of tuples): A list of (target, context) word pairs.
        """
        pairs = []
        for doc in corpus:
            tokens = self.preprocess(doc)
            for idx, word in enumerate(tokens):
                start = max(0, idx - self.window_size)
                end = min(len(tokens), idx + self.window_size + 1)
                for context_word in tokens[start:idx] + tokens[idx+1:end]:
                    pairs.append((word, context_word))
        return pairs
    
    def sigmoid(self, x):
        """
        Computes the sigmoid of x.
        
        Parameters:
        - x (float): The input value.
        
        Returns:
        - sigmoid (float): Sigmoid output.
        """
        return 1 / (1 + np.exp(-x))
    
    def train(self, corpus):
        """
        Trains the FastText model using the Skip-gram architecture with subword embeddings.
        
        Parameters:
        - corpus (list of str): A list of documents.
        """
        training_data = self.generate_training_data(corpus)
        subword_indices = {word: [self.subword_to_index[sub] for sub in self.get_subwords(word)] for word in self.vocab}
        
        for epoch in range(self.epochs):
            loss = 0
            for target_word, context_word in training_data:
                target_idx = self.word_to_index[target_word]
                context_idx = self.word_to_index[context_word]
                
                # Get subword indices for the target word
                target_subword_indices = subword_indices[target_word]
                
                # Compute target word embedding as the sum of its subword embeddings
                target_embedding = np.sum(self.W_sub[target_subword_indices], axis=0)
                
                # Positive sample
                context_embedding = self.W[context_idx]
                positive_score = self.sigmoid(np.dot(target_embedding, context_embedding))
                loss += -np.log(positive_score)
                
                # Gradient update for positive sample
                grad = self.learning_rate * (1 - positive_score)
                self.W[context_idx] += grad * target_embedding
                for sub_idx in target_subword_indices:
                    self.W_sub[sub_idx] += grad * context_embedding
                
                # Negative sampling
                for _ in range(self.negative_samples):
                    negative_idx = random.randint(0, len(self.vocab) - 1)
                    negative_embedding = self.W[negative_idx]
                    negative_score = self.sigmoid(-np.dot(target_embedding, negative_embedding))
                    loss += -np.log(negative_score)
                    
                    # Gradient update for negative sample
                    grad_neg = self.learning_rate * (1 - negative_score)
                    for sub_idx in target_subword_indices:
                        self.W_sub[sub_idx] -= grad_neg * negative_embedding
                    self.W[negative_idx] -= grad_neg * target_embedding
            
            print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {loss:.4f}")
    
    def get_embedding(self, word):
        """
        Retrieves the embedding for a given word, including OOV words.
        
        Parameters:
        - word (str): The target word.
        
        Returns:
        - embedding (numpy array): The word embedding vector.
        """
        if word in self.word_to_index:
            return self.W[self.word_to_index[word]]
        else:
            subword_indices = [self.subword_to_index[sub] for sub in self.get_subwords(word) if sub in self.subword_to_index]
            if subword_indices:
                return np.sum(self.W_sub[subword_indices], axis=0)
            else:
                raise ValueError(f"Word '{word}' not in vocabulary or subword list.")

I'm trying to break down when and how subword embeddings are updated during FastText training in a step-by-step in an intuitive way:

#### 1. Training in FastText is Word-Based

FastText works on word-level training pairs, similar to Word2Vec. However, subword embeddings are used to calculate the word embeddings.

For example, consider the sentence:

```
"The quick brown fox jumps over the lazy dog."
```

With a context window of 2, one Skip-gram training pair could be:

```
("quick","brown")
```

Here:

Target word: quick
Context word: brown

#### 2. Word Embeddings Are Composed of Subword Embeddings

**Subword N-grams**

Let’s decompose each word into its subword n-grams:

quick → `<qu`, `qui`, `uic`, `ick`, `k>`

brown → `<br`, `bro`, `row`, `own`, `wn>`

Each subword has its own **embedding vector**, initialized randomly.

**Compute Word Embeddings**

FastText computes the embedding of a word as the sum of its subword embeddings.

Embedding of `quick`:
```
embedding("quick")=embedding("<qu")+embedding("qui")+embedding("uic")+embedding("ick")+embedding("k>")
```

Embedding of `brown`:

```
embedding("brown")=embedding("<br")+embedding("bro")+embedding("row")+embedding("own")+embedding("wn>")
```

#### 3. Skip-gram Training Objective

The goal of Skip-gram is to maximize the probability of the context word (brown) given the target word (quick).

Mathematically:
```
P("brown"∣"quick")=σ(embedding("quick")⋅embedding("brown"))
```

Where, σ is the sigmoid function and ⋅ denotes the dot product.

This tells the model how likely brown appears in the context of quick.

#### 4. Loss and Gradient Calculation

The model computes a loss based on how well it predicted brown given quick.

- Step 1: Compute Prediction
    - Compute the dot product of the target (quick) and context (brown) embeddings.
    - Apply the sigmoid function to get the predicted probability.
- Step 2: Compute Error
    - Compare the predicted probability to the actual label (1 for correct pairs, 0 for negative samples).
    - The difference (error) tells the model how much it needs to adjust the embeddings.
- Step 3: Backpropagate the Error
    - The error is backpropagated to update both:
        - The subword embeddings of the target word (quick).
        - The subword embeddings of the context word (brown).

#### 5. Updating Subword Embeddings

The update process involves gradient descent.

Let’s focus on how the subwords of quick is updated:

Target Word: `quick`

The embedding of `quick` is composed of its subwords: `<qu`, `qui`, `uic`, `ick`, `k>`.

Each subword contributes to the overall embedding of quick. During backpropagation:

- The error calculated for quick is split among its subwords.
- Each subword’s embedding is updated based on its contribution.

For a subword s, its update rule looks like:

```
embedding(s)←embedding(s)+η⋅gradient
```
Where η is the learning rate. Gradient depends on the error and context embedding.

Context Word: `brown`
Similarly, the embedding of `brown` is composed of its subwords: `<br`, `bro`, `row`, `own`, `wn>`.

Each subword in brown is updated based on the error signal from the target word (`quick`).

#### 6. Intuition for Subword Updates

- Subwords are shared across multiple words.
    - For example, the subword `run` might appear in `running`, `runner`, and `runs`.
    - Every time one of these words is involved in a training pair, the subword `run` is updated.
    - Over time, frequent subwords (like `ing`) become well-optimized, while rare subwords improve slowly.